In [ ]:
# import os
# import torch
# import torch.distributed as dist
# from torch.utils.data import DataLoader

# import requests
# from datasets import load_dataset, Dataset
# from transformers import (
#     AutoTokenizer,
#     AutoModelForCausalLM,
#     BitsAndBytesConfig,
#     TrainingArguments,
#     get_scheduler,
# )
# from accelerate import init_empty_weights, infer_auto_device_map
# from peft import LoraConfig
# from trl import SFTTrainer
# from config import API_URL_2, HEADERS_2

: 

In [ ]:
import torch
from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline, TrainingArguments
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [ ]:
from datasets import load_dataset
dataset = load_dataset("daekeun-ml/naver-news-summarization-ko")

dataset
dataset['train'][0]

In [ ]:
# BASE_MODEL = "google/gemma-2b-it"

# model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map={"":0})
# tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, add_special_tokens=True)

In [ ]:
# doc = dataset['train']['document'][0]
# pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512)

# messages = [
#     {
#         "role": "user",
#         "content": "다음 글을 요약해주세요:\n\n{}".format(doc)
#     }
# ]
# prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

In [ ]:
# prompt

In [ ]:
# outputs = pipe(
#     prompt,
#     do_sample=True,
#     temperature=0.2,
#     top_k=50,
#     top_p=0.95,
#     add_special_tokens=True
# )
# print(outputs[0]["generated_text"][len(prompt):])

In [ ]:
def generate_prompt(example):
    prompt_list = []
    for i in range(len(example['document'])):
        prompt_list.append(r"""<bos><start_of_turn>user
다음 글을 요약해주세요:

{}<end_of_turn>
<start_of_turn>model
{}<end_of_turn><eos>""".format(example['document'][i], example['summary'][i]))
    return prompt_list

train_data = dataset['train']
print(generate_prompt(train_data[:1])[0])

In [ ]:
import torch
from peft import LoraConfig
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer

lora_config = LoraConfig(
    r=6,
    lora_alpha = 8,
    lora_dropout = 0.05,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype="float16",
#     bnb_4bit_use_double_quant=True
# )

BASE_MODEL = "google/gemma-2b-it"
# model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map="auto", quantization_config=bnb_config)
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=bnb_config,
    device_map=None,
    low_cpu_mem_usage=True,
    )
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, add_special_tokens=True)
tokenizer.padding_side = 'right'

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    max_seq_length=512,
    args=TrainingArguments(
        output_dir="outputs",
#        num_train_epochs = 1,
        max_steps=3000,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        optim="paged_adamw_8bit",
        warmup_steps=0.03,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=100,
        push_to_hub=False,
        report_to='none',
    ),
    peft_config=lora_config,
    formatting_func=generate_prompt,
)

In [ ]:
trainer.train()

In [ ]:
ADAPTER_MODEL = "lora_adapter"

trainer.model.save_pretrained(ADAPTER_MODEL)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map='auto', torch_dtype=torch.float16)
model = PeftModel.from_pretrained(model, ADAPTER_MODEL, device_map='auto', torch_dtype=torch.float16)

model = model.merge_and_unload()
model.save_pretrained('gemma-2b-it-sum-ko')

In [ ]:
# # pip install accelerate
# from transformers import AutoTokenizer, AutoModelForCausalLM

# tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b")
# model = AutoModelForCausalLM.from_pretrained("google/gemma-2b", device_map="auto")

# input_text = "Write me a poem about Machine Learning."
# input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

# outputs = model.generate(**input_ids)
# print(tokenizer.decode(outputs[0]))

In [ ]:
# def get_model_and_tokenizer(model_id):
#     tokenizer = AutoTokenizer.from_pretrained(model_id)
#     tokenizer.pad_token = tokenizer.eos_token
#     bnb_config = BitsAndBytesConfig(
#         load_in_4bit=True,
#         bnb_4bit_quant_type="nf4",
#         bnb_4bit_compute_dtype="float16",
#         bnb_4bit_use_double_quant=True
#     )
    
#     model = AutoModelForCausalLM.from_pretrained(
#         model_id,
#         quantization_config=bnb_config,
#         device_map=None,
#         low_cpu_mem_usage=True,
#     )
    
#     device_map = infer_auto_device_map(model, max_memory={0: "16GiB", "cpu": "32GiB"})
#     model = AutoModelForCausalLM.from_pretrained(
#         model_id,
#         quantization_config=bnb_config,
#         device_map=device_map,
#         low_cpu_mem_usage=True
#     )
    
#     model.config.use_cache = False
#     model.config.pretraining_tp = 1
#     model.config.hidden_activation = "gelu_pytorch_tanh"
#     model.gradient_checkpointing_enable()
#     return model, tokenizer

In [ ]:
# def preprocess_function(examples):
#     if "sentence2" in examples:
#         inputs = examples["sentence1"] + " [SEP] " + examples["sentence2"]
#     else:
#         inputs = examples["sentence"]
#     labels = examples["label"]
#     return {"input_text": inputs, "labels": labels}

In [ ]:
# def model_finetuning():
#     def query(payload):
#         response = requests.post(API_URL_2, headers=HEADERS_2, json=payload)
#         return response.json()
    
#     output = query({
#         "inputs": "Can you please let us know more details about your ",
#     })
#     print(output)

#     model_id = "google/gemma-2b"
#     model, tokenizer = get_model_and_tokenizer(model_id)

#     glue_dataset = load_dataset("glue", "sst2")

#     tokenized_dataset = glue_dataset.map(preprocess_function, batched=True)
#     tokenized_dataset = tokenized_dataset.map(
#         lambda examples: tokenizer(examples["input_text"], truncation=True, padding="max_length"),
#         batched=True,
#     )

#     # LoRA configuration
#     peft_config = LoraConfig(
#         r=8,  # Low-rank adaptation size
#         lora_alpha=16, 
#         lora_dropout=0.05, 
#         bias="none", 
#         task_type="CAUSAL_LM"
#     )

#     # Training arguments
#     training_args = TrainingArguments(
#         output_dir="./gemma-glue",
#         per_device_train_batch_size=1,  # Reduce batch size
#         gradient_accumulation_steps=32,
#         num_train_epochs=3,
#         logging_steps=10,
#         learning_rate=2e-4,
#         evaluation_strategy="steps",
#         save_steps=500,
#         save_total_limit=2,
#         fp16=True
#     )

#     trainer = SFTTrainer(
#         model=model,
#         train_dataset=tokenized_dataset["train"],
#         eval_dataset=tokenized_dataset["validation"],
#         tokenizer=tokenizer,
#         peft_config=peft_config,
#         args=training_args
#     )

#     trainer.train()

#     results = trainer.evaluate()
#     print(results)

#     trainer.save_model("./gemma2b-glue-finetuned")
#     tokenizer.save_pretrained("./gemma2b-glue-finetuned")


In [ ]:
# model_finetuning()